In [40]:
from glob import glob
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt

In [31]:
# Read all
ds_vs = xr.open_mfdataset('/storage/group/pches/default/users/dcl5300/gridMET/vs/*.nc')
ds_pr = xr.open_mfdataset('/storage/group/pches/default/users/dcl5300/gridMET/pr/*.nc')
ds_vpd = xr.open_mfdataset('/storage/group/pches/default/users/dcl5300/gridMET/vpd/*.nc')
ds_tmmx = xr.open_mfdataset('/storage/group/pches/default/users/dcl5300/gridMET/tmmx/*.nc')

In [32]:
# Ithaca lat/lon
lat, lon = 42.443333, -76.5

ds_vs_sel = ds_vs.sel(lat=lat, lon=lon, method='nearest').resample(day='YE').max()
ds_pr_sel = ds_pr.sel(lat=lat, lon=lon, method='nearest').resample(day='YE').max()
ds_vpd_sel = ds_vpd.sel(lat=lat, lon=lon, method='nearest').resample(day='YE').max()
ds_tmmx_sel = ds_tmmx.sel(lat=lat, lon=lon, method='nearest').resample(day='YE').max()

In [46]:
vs = ds_vs_sel['wind_speed'].to_dataframe().reset_index()
pr = ds_pr_sel['precipitation_amount'].to_dataframe().reset_index()
tmmx = ds_tmmx_sel['air_temperature'].to_dataframe().reset_index()
vpd = ds_vpd_sel['mean_vapor_pressure_deficit'].to_dataframe().reset_index()

In [48]:
df_merged = reduce(lambda  left, right: pd.merge(left, right, how='inner', on='day'), 
                   [vs.drop(columns=['lat','lon']), 
                    pr.drop(columns=['lat','lon']), 
                    tmmx.drop(columns=['lat','lon']), 
                    vpd.drop(columns=['lat','lon'])])

In [50]:
df_merged.to_csv('./data/ithaca_gridmet_annual_max.csv', index=False)

### Old

In [2]:
# import multiprocessing as mp
# print(mp.cpu_count())
# num_processes = mp.cpu_count() - 5

80


In [3]:
# TGW path
tgw_path = '/home/shared/vs498_0001/im3_hyperfacets_tgw'

In [4]:
# Get Ithaca lat/lon in TGW coords
lat, lon = 42.443333, -76.5

ds_tmp = salem.open_wrf_dataset(f'{tgw_path}/historical_1980_2019/hourly/tgw_wrf_historical_hourly_2006-09-03_01_00_00.nc')
ds_crs = ccrs.Projection(ds_tmp.pyproj_srs)
x, y = ds_crs.transform_point(lon, lat, src_crs=ccrs.PlateCarree())

In [5]:
# Function to process individual TGW file
def preprocess_tgw(file):
    # Variables to keep
    var_names = ['T2C', 'WSPD', 'PRCP', 'SFROFF', 'SNOW']
    try:
        # Read
        ds = salem.open_wrf_dataset(file)[var_names]
        # Select Ithaca location
        ds_sel = ds.sel({'west_east': x, 'south_north': y}, method="nearest", drop=True)
        # Resample to daily
        ds_sel_daily = xr.merge([
            ds_sel['T2C'].resample(time='1D').max(),
            ds_sel['WSPD'].resample(time='1D').max(),
            ds_sel['PRCP'].resample(time='1D').sum(),
            ds_sel['SFROFF'].resample(time='1D').sum(),
            ds_sel['SNOW'].resample(time='1D').sum(),
        ])
        # return 
        return ds_sel_daily
    except Exception as e:
        print(f"Error processing file {file}: {str(e)}")
        return None

In [6]:
# Function to process list of TGW files
def process_tgw_files(files):
    # Parallelize with mp
    with mp.Pool(processes=num_processes) as pool:
        # Map the worker function to all tasks
        results = pool.map(preprocess_tgw, np.sort(files)[:50])
            
        # Filter out None results
        ds_all = [ds for ds in results if ds is not None]

    # Calculate annual maxima and store
    df = xr.concat(ds_all, dim='time').resample(time='YE').max().to_dataframe()
    df = df.drop(columns=['lat', 'lon', 'west_east', 'south_north']).reset_index()
    return df

In [7]:
%%time
# Process TGW historical
scenario = 'historical_1980_2019'
files = glob(f'{tgw_path}/{scenario}/hourly/*.nc')

df = process_tgw_files(files)

CPU times: user 101 ms, sys: 224 ms, total: 325 ms
Wall time: 5min 6s


In [8]:
df

,time,T2C,WSPD,PRCP,SFROFF,SNOW
0,1980-12-31,32.4263,16.602165,38.049194,6140.94043,706.901611


In [ ]:
%%time
# Process TGW future scenarios
scenarios = ['rcp45cooler', 'rcp45hotter', 'rcp85cooler', 'rcp85hotter']
for scenario in scenarios:
    # Get all file names
    folders = glob(f'{tgw_path}/{scenario}*')
    files = np.array([glob(f'{folder}/hourly/*.nc') for folder in folders]).flatten()
    # Compute
    df = process_tgw_files(files)